### Tensor Flow 2-11 notebook

Notes: 
- Notebook should be running with an Nvidia GPU to for top performance
- load_img will load a nparray with x, y, 3 color dims.  
- If we're just processing one image we will need to np.expand_dims(image, axies=0) to get 1, x, y, 3. The library expects N images in the first dim
- should set random seed on tf.random.set_seed(1) for reproduceability 
- 

last change 3/6/2025

In [6]:
# import the libraries for training, testing, validation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import imagenet_utils  # will decode predictions out of the model into a 4 dim array of N (image num), imageID, label, probability result[0] would be the set of results for image one
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img  # will load img and reshape, usage is load_img(image_name_loc, target_size=input_shape)
from tensorflow.keras.utils import plot_model  # Note: usage syntax is plot_model(instantied_model, to_file='', show_shapes=True)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds  # For loading datasets from GCS


import matplotlib.pyplot as plt
import numpy as np
import os
import time


2025-03-11 13:17:58.518461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 13:18:01.300294: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-03-11 13:18:01.300457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

In [ ]:
# Configuration
GCS_BUCKET = 'nabirds_filtered'  
DATASET_PATH = 'images'  # Relative path within the bucket
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = None  # determined from the dataset

In [7]:
# define a dictionary for model config and experiment tracking.... 
model_input_variables = {
    "EfficientNetB0": {
        "image_size": (224, 224),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "MobileNetV2": {
        "image_size": (224, 224),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
     "MobileNetV3Large": { #MobileNetV3 Large will be used to compare accuracy loss with small, prob not a candidate for rasp pi running
        "image_size": (224, 224),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "MobileNetV3Small":{ 
        "image_size": (224, 224),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "InceptionV3": {
        "image_size": (299, 299),  # InceptionV3 typically uses 299x299
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "EfficientNetB7":{
        "image_size": (600,600),  # big values!
        "batch_size": 16, # this is a larger model and the norm seems to be smaller batch sizes for reduced memory use
        "epochs": 10,
        "learning_rate": 0.0001,
    }
}

print(f'EfficientNetB0: {model_input_variables["EfficientNetB0"]}')
print(f'MobileNetV2: {model_input_variables["MobileNetV2"]}')
print(f'MobileNetV3Large: {model_input_variables["MobileNetV3Large"]}')
print(f'MobileNetV3Small: {model_input_variables["MobileNetV3Small"]}')
print(f'InceptionV3: {model_input_variables["InceptionV3"]}')
print(f'EfficientNetB7: {model_input_variables["EfficientNetB7"]}')

EfficientNetB0: {'image_size': (224, 224), 'batch_size': 32, 'epochs': 10, 'learning_rate': 0.0001}
MobileNetV2: {'image_size': (224, 224), 'batch_size': 32, 'epochs': 10, 'learning_rate': 0.0001}
MobileNetV3Large: {'image_size': (224, 224), 'batch_size': 32, 'epochs': 10, 'learning_rate': 0.0001}
MobileNetV3Small: {'image_size': (224, 224), 'batch_size': 32, 'epochs': 10, 'learning_rate': 0.0001}
InceptionV3: {'image_size': (299, 299), 'batch_size': 32, 'epochs': 10, 'learning_rate': 0.0001}
EfficientNetB7: {'image_size': (600, 600), 'batch_size': 16, 'epochs': 10, 'learning_rate': 0.0001}


In [ ]:
def load_gcs_dataset(bucket_name, dataset_path, image_size, batch_size):
    """Loads a dataset from Google Cloud Storage."""
    dataset = None
    gcs_dataset_path = f"gs://{bucket_name}/{dataset_path}"
    try:
        # Attempt to load the dataset using tf.keras.utils.image_dataset_from_directory.
        dataset = keras.utils.image_dataset_from_directory(
            gcs_dataset_path,
            image_size=image_size,
            batch_size=batch_size,
            label_mode='categorical',)  # categorical is for softmax layer
    except Exception as e:
        print(f"Error loading dataset from GCS: {e}")
        
    return dataset


In [ ]:
train_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'train'), IMAGE_SIZE, BATCH_SIZE)
validation_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'test'), IMAGE_SIZE, BATCH_SIZE)

if train_dataset is None or validation_dataset is None:
    print("Dataset loading failed. Exiting.")
    exit()
num_classes = len(train_dataset.class_names) # Get class count.

In [6]:
# ?? code needs to be changed for experiment tracking and different model types

##### MobileNetV2 training sample, use for comparision to old model in feeder
start_time = time.time()
print(start_time)

###### load mobilenetv2 and modify head
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Unlock the base model, note this should be changed to freeze some layers per lit review
epochs = 10  

# add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional: Add more dense layers
predictions = Dense(num_classes, activation='softmax')(x)

# create the model with Rescaling layer
inputs = base_model.input
rescaled_inputs = tf.keras.layers.Rescaling(1./255)(inputs)  # normalize pixel values to [0, 1], not sure if this was done in the old mobilenetv2 model
x = base_model(rescaled_inputs) # pass the rescaled input through the base model.
model = Model(inputs=inputs, outputs=predictions) # use the original inputs.

# compile model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',  # Important for multi-class classification
              metrics=['accuracy'])

# train the model
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset
)

# save model
model.save('mobilenet_retrained.h5')
print('training complete at {time.time()}.')
print(f'elapsed time in minutes: {(time.time() - start_time)/60:.2f}')

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10


2025-03-09 14:28:27.768350: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 92 of 256
2025-03-09 14:28:37.855424: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 187 of 256
2025-03-09 14:28:45.256797: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - ETA: 0s - loss: 1.9390 - accuracy: 0.4908

2025-03-09 14:33:40.395498: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 92 of 256
2025-03-09 14:33:50.414424: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 189 of 256
2025-03-09 14:33:57.230597: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - 391s 5s/step - loss: 1.9390 - accuracy: 0.4908 - val_loss: 2.7848 - val_accuracy: 0.1965
Epoch 2/10


...

77/77 [==============================] - 345s 4s/step - loss: 0.0119 - accuracy: 0.9992 - val_loss: 0.6622 - val_accuracy: 0.7971
Epoch 10/10


2025-03-09 15:30:31.924979: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 116 of 256
2025-03-09 15:30:41.857610: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 231 of 256
2025-03-09 15:30:44.146693: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - ETA: 0s - loss: 0.0078 - accuracy: 0.9996

2025-03-09 15:35:21.684198: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 114 of 256
2025-03-09 15:35:31.718052: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 236 of 256
2025-03-09 15:35:33.325738: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - 348s 4s/step - loss: 0.0078 - accuracy: 0.9996 - val_loss: 0.6094 - val_accuracy: 0.8068
Training complete.
